## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,0,Rikitea,-23.1203,-134.9692,75.90,74,99,16.17,overcast clouds,PF,2021-12-11 20:19:04
1,1,Victoria,22.2855,114.1577,68.32,87,0,3.00,clear sky,HK,2021-12-11 20:14:15
2,2,Sao Felix Do Xingu,-6.6447,-51.9950,78.78,94,100,1.90,overcast clouds,BR,2021-12-11 20:19:04
3,3,Kodinsk,58.6881,99.1844,19.18,92,100,7.40,overcast clouds,RU,2021-12-11 20:19:04
4,4,Iqaluit,63.7506,-68.5145,-11.31,69,20,5.01,few clouds,CA,2021-12-11 20:19:05


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 75


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
pclen = len(preferred_cities_df)
print(pclen)
preferred_cities_df.head(10)

222


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
1,1,Victoria,22.2855,114.1577,68.32,87,0,3.00,clear sky,HK,2021-12-11 20:14:15
6,6,Ihosy,-22.4000,46.1167,69.55,84,98,9.51,overcast clouds,MG,2021-12-11 20:19:05
13,13,Port Alfred,-33.5906,26.8910,69.91,89,100,6.80,light rain,ZA,2021-12-11 20:19:07
14,14,Pisco,-13.7000,-76.2167,73.45,68,0,17.27,clear sky,PE,2021-12-11 20:19:07
15,15,Albany,42.6001,-73.9662,59.81,91,100,7.92,overcast clouds,US,2021-12-11 20:16:47
18,18,Busselton,-33.6500,115.3333,57.04,84,69,14.54,broken clouds,AU,2021-12-11 20:19:08
24,24,Kaitangata,-46.2817,169.8464,53.92,58,76,1.97,broken clouds,NZ,2021-12-11 20:19:10
27,27,Hermanus,-34.4187,19.2345,63.90,76,84,13.71,broken clouds,ZA,2021-12-11 20:17:23
28,28,Ushuaia,-54.8000,-68.3000,62.26,45,40,3.44,scattered clouds,AR,2021-12-11 20:14:18
29,29,Olean,42.0776,-78.4297,64.13,94,90,26.46,overcast clouds,US,2021-12-11 20:19:11


In [12]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.empty)

False


In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows
preferred_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country,Date
0,0,Rikitea,-23.1203,-134.9692,75.90,74,99,16.17,overcast clouds,PF,2021-12-11 20:19:04
1,1,Victoria,22.2855,114.1577,68.32,87,0,3.00,clear sky,HK,2021-12-11 20:14:15
2,2,Sao Felix Do Xingu,-6.6447,-51.9950,78.78,94,100,1.90,overcast clouds,BR,2021-12-11 20:19:04
5,5,Padang,-0.9492,100.3543,76.98,88,95,4.56,overcast clouds,ID,2021-12-11 20:15:28
6,6,Ihosy,-22.4000,46.1167,69.55,84,98,9.51,overcast clouds,MG,2021-12-11 20:19:05
...,...,...,...,...,...,...,...,...,...,...,...
678,678,Tual,-5.6667,132.7500,84.07,75,100,13.82,overcast clouds,ID,2021-12-11 20:36:24
679,679,Umm Lajj,25.0213,37.2685,75.90,52,100,2.28,overcast clouds,SA,2021-12-11 20:36:25
680,680,Garowe,8.4054,48.4845,71.87,70,18,14.61,few clouds,SO,2021-12-11 20:37:25
683,683,Marawi,7.9986,124.2928,66.65,94,77,1.90,broken clouds,PH,2021-12-11 20:37:26


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Victoria,HK,68.32,clear sky,22.2855,114.1577,
6,Ihosy,MG,69.55,overcast clouds,-22.4000,46.1167,
13,Port Alfred,ZA,69.91,light rain,-33.5906,26.8910,
14,Pisco,PE,73.45,clear sky,-13.7000,-76.2167,
15,Albany,US,59.81,overcast clouds,42.6001,-73.9662,
18,Busselton,AU,57.04,broken clouds,-33.6500,115.3333,
24,Kaitangata,NZ,53.92,broken clouds,-46.2817,169.8464,
27,Hermanus,ZA,63.90,broken clouds,-34.4187,19.2345,
28,Ushuaia,AR,62.26,scattered clouds,-54.8000,-68.3000,
29,Olean,US,64.13,overcast clouds,42.0776,-78.4297,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Victoria,HK,68.32,clear sky,22.2855,114.1577,Mini Hotel Central
6,Ihosy,MG,69.55,overcast clouds,-22.4000,46.1167,Hotely Sariaka
13,Port Alfred,ZA,69.91,light rain,-33.5906,26.8910,The Halyards Hotel
14,Pisco,PE,73.45,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
15,Albany,US,59.81,overcast clouds,42.6001,-73.9662,
...,...,...,...,...,...,...,...
672,Gobabis,NaN,71.06,moderate rain,-22.4500,18.9667,Goba Lodge
675,Turayf,SA,52.05,clear sky,31.6725,38.6637,Turaif
676,Henties Bay,NaN,60.85,scattered clouds,-22.1160,14.2845,Fishermans Guest House
680,Garowe,SO,71.87,few clouds,8.4054,48.4845,Curubo Hotel


In [26]:
hotel_df["Hotel Name"].replace('', np.nan, inplace=True)

In [25]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Victoria,HK,68.32,clear sky,22.2855,114.1577,Mini Hotel Central
6,Ihosy,MG,69.55,overcast clouds,-22.4000,46.1167,Hotely Sariaka
13,Port Alfred,ZA,69.91,light rain,-33.5906,26.8910,The Halyards Hotel
14,Pisco,PE,73.45,clear sky,-13.7000,-76.2167,Hotel San Isidro Oficial
18,Busselton,AU,57.04,broken clouds,-33.6500,115.3333,Observatory Guest House
...,...,...,...,...,...,...,...
672,Gobabis,NaN,71.06,moderate rain,-22.4500,18.9667,Goba Lodge
675,Turayf,SA,52.05,clear sky,31.6725,38.6637,Turaif
676,Henties Bay,NaN,60.85,scattered clouds,-22.1160,14.2845,Fishermans Guest House
680,Garowe,SO,71.87,few clouds,8.4054,48.4845,Curubo Hotel


In [30]:
# 8a. Create the output File (CSV)
output_data_file = "Vaction_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))